# AURORA 
Segmenting 

## Setup

In [1]:
!pip install brainles_aurora

## Data

Aurora expects preprocessed input data as NIfTI file or NumPy Array.  
If your data is not preprocessed yet consider using our [BrainLes preprocessing](https://github.com/BrainLesion/preprocessing) package (or until it is ready [BraTS-Toolkit](https://github.com/neuronflow/BraTS-Toolkit)).

In this example we provide sample data from the [ASNR-MICCAI BraTS Brain Metastasis Challenge](https://www.synapse.org/#!Synapse:syn51156910/wiki/622553), which is already preprocessed.

## Using Aurora

### Minimal example using default settings and only T1 as input

In [2]:
from brainles_aurora.inferer import AuroraGPUInferer, AuroraInferer, AuroraInfererConfig

config = AuroraInfererConfig()  # Use default config

# If you don-t have a GPU that supports CUDA use the CPU version: AuroraInferer(config=config)
inferer = AuroraGPUInferer(config=config)

inferer.infer(
    t1="data/t1n.nii.gz",
    segmentation_file="output/segmentation.nii.gz",
)

2024-01-29 12:27:46 INFO: Initialized AuroraGPUInferer with config: AuroraInfererConfig(output_mode=<DataMode.NIFTI_FILE: 'NIFTI_FILEPATH'>, log_level=20, tta=True, sliding_window_batch_size=1, workers=0, threshold=0.5, sliding_window_overlap=0.5, crop_size=(192, 192, 32), model_selection=<ModelSelection.BEST: 'best'>)


AssertionError: No cuda device available while using GPUInferer

### Advanced usage

In [ ]:
from brainles_aurora.inferer import AuroraGPUInferer, AuroraInferer, AuroraInfererConfig

config = AuroraInfererConfig(
    
)

# If you don-t have a GPU that supports CUDA use the CPU version: AuroraInferer(config=config)
inferer = AuroraGPUInferer(config=config)

inferer.infer(
    t1="data/t1n.nii.gz",
    segmentation_file="output/segmentation.nii.gz",
)